# How to develop and debug a Polarion LiveDoc

Welcome to the capella2polarion notebook where local development and debugging 
of a Polarion LiveDoc is showcased. This notebook will show you the following:
- How to load all current Polarion work-items into the PolarionWorker
- How to load all current Polarion LiveDocs under a specified space
- How to create and update Polarion LiveDocs under a specified space

Before we can interact with the REST API of Polarion we need to prepare our
environment:
Create a `.env` file with the following values:
- POLARION_PROJECT
- POLARION_HOST
- POLARION_PAT
- MODEL_PATH

First the load the Capella model with capellambse:

In [ ]:
from capella2polarion.connectors import polarion_worker
from capella2polarion.converters import document_renderer, document_config

import dotenv
import os
import capellambse
import pathlib

dotenv.load_dotenv()

test_data_path = pathlib.Path("../../../tests/data")

model = capellambse.MelodyModel(os.environ.get(
    "MODEL_PATH") or str(test_data_path / "model/Melody Model Test.aird")
)
worker = polarion_worker.CapellaPolarionWorker(
    polarion_worker.PolarionWorkerParams(
            os.environ.get("POLARION_PROJECT") or "",
            os.environ.get("POLARION_HOST") or "",
            os.environ.get("POLARION_PAT") or "",
            delete_work_items=False,
        )
)

## How to load all current work items from the Polarion project into the worker:

In [ ]:
worker.load_polarion_work_item_map()

Now we load all current LiveDocs under a specific space. This is configured in
the document config `capella2polarion_document_config.yaml`:

In [7]:
document_rendering_config_path = pathlib.Path("configs/capella2polarion_document_config.yaml.j2")
print(document_rendering_config_path.read_text(encoding="utf8"))

# Copyright DB InfraGO AG and contributors
# SPDX-License-Identifier: Apache-2.0

full_authority:
  - template_directory: jinja/document_templates
    template: test.html.j2
    heading_numbering: True
    work_item_layouts:
      componentExchange:
        fields_at_start:
          - id
        fields_at_end:
          - context_diagram
      physicalLink:
        fields_at_start:
          - id
      physicalActor:
        fields_at_start:
          - id
      physicalComponentActor:
        fields_at_start:
          - id
      physicalComponentNode:
        fields_at_start:
          - id
      physicalComponentBehavior:
        fields_at_start:
          - id
      class:
        show_title: False
        fields_at_end:
          - tree_view
      exchangeItem:
        show_title: False
    instances:
    {%- for pc in model.search("PhysicalComponent") %}
    {%- if pc.components %}
    - polarion_space: PC-Drafts
      polarion_name: {{ pc.uuid }}
      polarion_title: "{{ pc.na

If you want to know more about the features and limitations, head into the
documentation of the configuration for live doc rendering.

## How to load all current Polarion LiveDocs under a specified space

In [ ]:
renderer = document_renderer.DocumentRenderer(
    worker.polarion_data_repo,
    model,
    os.environ.get("POLARION_PROJECT") or "",
    overwrite_heading_numbering=True,
    overwrite_layouts=True,
)
with document_rendering_config_path.open("r", encoding="utf8") as file:
    configs = document_config.read_config_file(file, model)

documents = worker.load_polarion_documents(configs.iterate_documents())

## How to create and update Polarion LiveDocs under a specified space

In [ ]:
projects_document_data = renderer.render_documents(configs, documents)
for project, project_data in projects_document_data.items():
    worker.create_documents(project_data.new_docs, project)
    worker.update_documents(project_data.updated_docs, project)